# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [47]:
import pandas as pd
import numpy as np
import re

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [48]:
def answer_one():
    import pandas as pd
    import numpy as np
    import re
    
    # Energy DatdFrame
    df=pd.read_excel("assets/Energy Indicators.xls")


    Energy=df.iloc[17:244,[1, 2, 3, 4, 5]].drop('Unnamed: 2', 1).rename(columns={'Unnamed: 1':'Country',
                                                                                 'Unnamed: 3':'Energy Supply',
                                                                                 'Unnamed: 4':'Energy Supply per Capita',
                                                                                 'Unnamed: 5':'% Renewable'}).replace({'Energy Supply' : '...',
                                                                                                                       'Energy Supply per Capita' : '...'}, np.NaN).reset_index().drop('index', axis=1)
    
    Energy['Energy Supply']= Energy['Energy Supply'].apply(lambda x :  x * (1000000) )
    
    Energy['Country']=Energy['Country'].replace({"Republic of Korea": "South Korea",
                                                 "United States of America": "United States",
                                                 "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                                 "China, Hong Kong Special Administrative Region": "Hong Kong",
                                                 "Iran (Islamic Republic of)": "Iran",
                                                 "Bolivia ": "Bolivia",
                                                 "Falkland Islands ": "Falkland Islands",
                                                 "Micronesia  ": "Micronesia",
                                                 "Sint Maarten  ": "Sint Maarten",
                                                 "Venezuela ": "Venezuela"})
    
    pattern= '\([^)]+\)'
 
    Energy['Country'] = Energy['Country'].replace(to_replace = pattern, value = '', regex = True)
 
    Energy['Country'] = Energy['Country'].str.replace(r"\d*","")
    # strip leading and trailing space 
    Energy['Country'] = Energy['Country'].str.strip()
    
    #GDP DataFrame
    
    df1=pd.read_csv("assets/world_bank.csv", header=4)
    
    df1['Country Name']=df1['Country Name'].replace({'Korea, Rep.': 'South Korea',
                                                     'Iran, Islamic Rep.': 'Iran',
                                                     'Hong Kong SAR, China': 'Hong Kong',
                                                     'Macao SAR, China': 'China, Macao Special Administrative Region',
                                                     'Micronesia, Fed. Sts.': 'Micronesia',
                                                     'Sint Maarten (Dutch part)': 'Sint Maarten',
                                                     'Venezuela, RB': 'Venezuela'})
    
    GDP=df1[['Country Name','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015']].rename(columns={'Country Name':'Country'})
    
    # strip leading and trailing space 
    GDP['Country'] = GDP['Country'].str.strip()
    
    #ScimEn DataFrame
    ScimEn=pd.read_excel("assets/scimagojr-3.xlsx")
    
    # strip leading and trailing space 
    ScimEn['Country'] = ScimEn['Country'].str.strip()

    # merge all the three dataframes Energy, GDP and ScimEn,
    
    final_df = ScimEn.merge(Energy,how="inner",left_on="Country",right_on="Country").merge(GDP,how="inner",left_on="Country",right_on="Country").set_index('Country')
    
    # Select the first 15 rows of the Dataframe
    Energy_rank = final_df.head(15)

    return Energy_rank

In [49]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [50]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [51]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [52]:
def answer_two():
    import pandas as pd
    import numpy as np
    import re
    
    # Energy DatdFrame
    df=pd.read_excel("assets/Energy Indicators.xls")


    Energy=df.iloc[17:244,[1, 2, 3, 4, 5]].drop('Unnamed: 2', 1).rename(columns={'Unnamed: 1':'Country',
                                                                                 'Unnamed: 3':'Energy Supply',
                                                                                 'Unnamed: 4':'Energy Supply per Capita',
                                                                                 'Unnamed: 5':'% Renewable'}).replace({'Energy Supply' : '...',
                                                                                                                       'Energy Supply per Capita' : '...'}, np.NaN).reset_index().drop('index', axis=1)
    
    Energy['Energy Supply']= Energy['Energy Supply'].apply(lambda x :  x * (1000000) )
    
    Energy['Country']=Energy['Country'].replace({"Republic of Korea": "South Korea",
                                                 "United States of America": "United States",
                                                 "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                                 "China, Hong Kong Special Administrative Region": "Hong Kong",
                                                 "Iran (Islamic Republic of)": "Iran"})
    
    pattern= '\([^)]+\)'
 
    Energy['Country'] = Energy['Country'].replace(to_replace = pattern, value = '', regex = True)
 
    Energy['Country'] = Energy['Country'].str.replace(r"\d*","")
    # strip leading and trailing space 
    Energy['Country'] = Energy['Country'].str.strip()
    
    #GDP DataFrame
    
    df1=pd.read_csv("assets/world_bank.csv", header=4)
    
    df1['Country Name']=df1['Country Name'].replace({'Korea, Rep.': 'South Korea',
                                                     'Iran, Islamic Rep.': 'Iran',
                                                     'Hong Kong SAR, China': 'Hong Kong',
                                                     'Macao SAR, China': 'China, Macao Special Administrative Region',
                                                     'Micronesia, Fed. Sts.': 'Micronesia',
                                                     'Sint Maarten (Dutch part)': 'Sint Maarten',
                                                     'Venezuela, RB': 'Venezuela'})
    
    GDP=df1[['Country Name','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015']].rename(columns={'Country Name':'Country'})
    
    # strip leading and trailing space 
    GDP['Country'] = GDP['Country'].str.strip()

    #ScimEn DataFrame
    ScimEn=pd.read_excel("assets/scimagojr-3.xlsx")
    
    # strip leading and trailing space 
    ScimEn['Country'] = ScimEn['Country'].str.strip()

    outer_merge = ScimEn.merge(Energy,how="outer",left_on="Country",right_on="Country").merge(GDP,how="outer",left_on="Country",right_on="Country").set_index('Country')
    final_df = ScimEn.merge(Energy,how="inner",left_on="Country",right_on="Country").merge(GDP,how="inner",left_on="Country",right_on="Country").set_index('Country')

    rows_dropped =  outer_merge.shape[0] - final_df.shape[0]

    return rows_dropped

In [53]:
answer_two()

151

In [54]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [55]:
def answer_three():
    
    import numpy as np
    
    # Apply function sum() to columns 2006 to 2015 for each country
    #create series 'avgGDP'
    
    Energy_rank = answer_one()
    
    avgGDP = Energy_rank.iloc[:][['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']].rename(columns={'Country Name':'Country'}).apply(np.mean,axis = 1).sort_values(ascending = False)

    return avgGDP

In [56]:
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [57]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [58]:
def answer_four():
    
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    Energy_rank['avgGDP']= Energy_rank.iloc[:][['2006','2007','2008','2009','2010',
                                                '2011','2012','2013','2014','2015']].rename(columns={'Country Name':'Country'}).apply(np.mean,axis = 1).sort_values(ascending = False)


    Energy_rank=Energy_rank.sort_values('avgGDP', ascending = False)
   
    return Energy_rank.iloc[5]['2015']-Energy_rank.iloc[5]['2006']

In [59]:
answer_four()

246702696075.3999

In [60]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [61]:
def answer_five():
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    return Energy_rank['Energy Supply per Capita'].mean(axis = 0)

In [62]:
answer_five()

157.6

In [63]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [64]:
def answer_six():
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    return Energy_rank['% Renewable'].idxmax(), Energy_rank['% Renewable'].max()

In [65]:
answer_six()

('Brazil', 69.64803)

In [66]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [67]:
def answer_seven():
    
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    Energy_rank['Ratio_SC_TC']= Energy_rank.apply(lambda row: row['Self-citations']/row['Citations'], axis=1)
    
    return Energy_rank['Ratio_SC_TC'].idxmax(), Energy_rank['Ratio_SC_TC'].max()

In [68]:
answer_seven()

('China', 0.6893126179389422)

In [69]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [70]:
def answer_eight():
    
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    Energy_rank['Population']= Energy_rank.apply(lambda row: row['Energy Supply']/row['Energy Supply per Capita'], axis=1)
    
    return Energy_rank['Population'].sort_values(ascending = False).index[2]

In [71]:
answer_eight()

'United States'

In [72]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [73]:
def answer_nine():
    
    import pandas as pd
    import numpy as np
    import scipy.stats as stats

    Energy_rank = answer_one()
    
    Energy_rank['Population']= Energy_rank['Energy Supply']/Energy_rank['Energy Supply per Capita']
     
    Energy_rank['Citations per Capita']= Energy_rank['Citable documents']/Energy_rank['Population']
    
    corr_df=Energy_rank[['Citations per Capita','Energy Supply per Capita']]

    # here is some stub code to actually run the correlation
    corr, pval=stats.pearsonr(corr_df["Citations per Capita"],corr_df["Energy Supply per Capita"])
    
    return corr

In [74]:
answer_nine()

0.7940010435442943

In [75]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [76]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [77]:
def answer_ten():
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    median=Energy_rank['% Renewable'].sort_values().iloc[7]
    
    Energy_rank['status_renewable'] = [1 if x >= median else 0 for x in Energy_rank['% Renewable']]

    return Energy_rank.sort_values('Rank').iloc[:]['status_renewable']

In [78]:
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: status_renewable, dtype: int64

In [79]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [80]:
def answer_eleven():
    
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    Energy_rank['Population']= Energy_rank.apply(lambda row: row['Energy Supply']/row['Energy Supply per Capita'], axis=1)

    ContinentDict  = {'China':'Asia',
                      'United States':'North America',
                      'Japan':'Asia',
                      'United Kingdom':'Europe',
                      'Russian Federation':'Europe',
                      'Canada':'North America',
                      'Germany':'Europe',
                      'India':'Asia',
                      'France':'Europe',
                      'South Korea':'Asia',
                      'Italy':'Europe',
                      'Spain':'Europe',
                      'Iran':'Asia',
                      'Australia':'Australia',
                      'Brazil':'South America'}

    new_datadrame = pd.DataFrame(ContinentDict.items(), columns=['Country', 'Continent'])

    datadrame_popultion = pd.merge(new_datadrame,Energy_rank['Population'],on='Country', how='left')
    
    final_df=datadrame_popultion.groupby(by=['Continent']).agg({'Population':['size', 'sum', 'mean', 'std']}).droplevel(0, axis=1)
    
    return final_df

In [81]:
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [82]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [83]:
def answer_twelve():
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    ContinentDict  = {'China':'Asia',
                      'United States':'North America',
                      'Japan':'Asia',
                      'United Kingdom':'Europe',
                      'Russian Federation':'Europe',
                      'Canada':'North America',
                      'Germany':'Europe',
                      'India':'Asia',
                      'France':'Europe',
                      'South Korea':'Asia',
                      'Italy':'Europe',
                      'Spain':'Europe',
                      'Iran':'Asia',
                      'Australia':'Australia',
                      'Brazil':'South America'}

    new_datadrame = pd.DataFrame(ContinentDict.items(), columns=['Country', 'Continent'])
    
    datadrame_renewable = pd.merge(new_datadrame,Energy_rank['% Renewable'],on='Country', how='left')
    
    datadrame_renewable['% Renewable']=pd.cut(datadrame_renewable['% Renewable'], bins=5)
    
    return datadrame_renewable.groupby(['Continent','% Renewable'])['Continent'].agg(np.size).dropna()

In [84]:
answer_twelve()

Continent      % Renewable     
Asia           (2.212, 15.753]     4.0
               (15.753, 29.227]    1.0
Australia      (2.212, 15.753]     1.0
Europe         (2.212, 15.753]     1.0
               (15.753, 29.227]    3.0
               (29.227, 42.701]    2.0
North America  (2.212, 15.753]     1.0
               (56.174, 69.648]    1.0
South America  (56.174, 69.648]    1.0
Name: Continent, dtype: float64

In [85]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [86]:
def answer_thirteen():
    import pandas as pd
    import numpy as np
    
    Energy_rank = answer_one()
    
    Energy_rank['Population']= Energy_rank.apply(lambda row: row['Energy Supply']/row['Energy Supply per Capita'], axis=1)
    
    PopEst= Energy_rank['Population'].apply('{:,}'.format)
    
    return PopEst

In [87]:
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: Population, dtype: object

In [88]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [89]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")